# Usecase

<p>This code was used to generate evolutionary scores for all features in the dataset.</p>

<ol>
  <li>combined_binary_mutations_non_snp_corrected_0.2.tsv - the dataset with all features</li>
  <li>run_all_LSTINFO-LSTINFO-NA-filtered-0.0_0.2.lst - panacota generated LSTINFO file</li>
  <li>MYTU_4648.nucl.grp.aln.iqtree_tree.treefile - panacota IQTREE generated tree file</li>

</ol>

In [ ]:
from utils import *
from score import *


with open('../../mtb/MYTU_4648.nucl.grp.aln.iqtree_tree.treefile', "r") as f:
    newick=f.readlines()[0]

t, node_list, names_list = load_tree(newick)
#get_distance_matrix(t, node_list, names_list)

dist_matrix = pd.read_csv("dist_matrix_all_incl_internal.csv", index_col="Unnamed: 0")
dist_matrix = dist_matrix.fillna(0)



featurefile="../../mtb/new_data_feb4/combined_binary_mutations_non_snp_corrected_0.2.tsv"
#bact_dataloc="/nfs/scistore08/kondrgrp/sbuyanov/muts_in_bact"
bact_dataloc="../"
mtb_datafile="run_all_LSTINFO-LSTINFO-NA-filtered-0.0_0.2.lst" # panacota LSTINFO output file
mtb_id_match=load_renaming_and_features(bact_dataloc, mtb_datafile, featurefile,origformat=True)


mtb_id_match=mtb_id_match.replace("?",2)
mtb_id_match = mtb_id_match.fillna(0)
#convert to numbers, since "?" made it be imported as str
mtb_id_match.iloc[:,3:]=mtb_id_match.iloc[:,3:].apply(pd.to_numeric)

feature_score_dict={}
empty_f=[]



feature_score_dict={}

#for it in range(1,2):
#    l=1000*it
for feature in tqdm(mtb_id_match.columns):#[3+l-1000:l]):
    samples=mtb_id_match.loc[mtb_id_match[feature]==1,"panacota_renamed"] #select those who have the feature           

    if len(samples)==0:
        empty_f.append(feature)
    else:
        feature_score_dict[feature]=find_nodes_with_descendants_from_matrix(samples, t, dist_matrix)
        
feature_score=pd.DataFrame.from_dict(feature_score_dict,orient="index")

feature_score=feature_score.sum(axis=1)
feature_score.columns = ["score"]
feature_score.to_csv(f"new_score_from_matrix_all.csv")
